In [ ]:
import re
import time

import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [ ]:
wd = webdriver.Chrome(ChromeDriverManager().install())
years = range(1964, 2021)
song_id_regex = re.compile("[0-9]+")
base_url = (
    "https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate="
)
songList = pd.DataFrame(columns=["year", "title", "artist", "song_id"])


In [ ]:
for year in years:
    wd.get(base_url + str(year))

    chart = wd.find_element(By.CSS_SELECTOR, "form#frm").find_element(
        By.CSS_SELECTOR, "form#frm table"
    )
    current = pd.DataFrame(
        {
            "year": year,
            "title": [
                title.get_attribute("textContent").strip()
                for title in chart.find_elements(
                    By.CSS_SELECTOR, "div.wrap_song_info div.rank01"
                )
            ],
            "artist": [
                artist.get_attribute("textContent")
                for artist in (
                    chart.find_elements(By.CSS_SELECTOR, "span.checkEllipsis")
                )
            ],
            "song_id": [
                song_id_regex.findall(song_id.get_attribute("onclick"))[0]
                for song_id in (chart.find_elements(By.CSS_SELECTOR, "div.wrap a.btn"))
            ],
        }
    )
    songList = pd.concat((songList, current))

    time.sleep(3)


In [1]:
songList.to_csv(f"../data/lyrics_original/kpop_lyrics_crawling_step1.csv", encoding="utf-8-sig", index=False)

NameError: name 'songList' is not defined